In [4]:
from fyers_apiv3 import fyersModel
import pandas as pd
import datetime

# Initialize the Fyers API client
access_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb1BvbHVaaHMxYXdwZjR1eUlPcnFjQTFfQVNzbzlDSXMxYTExdzQ2akZMbDBNSjlJSXhmUmE3THEzT0xmSVl2XzZZY1RTZFNQMkd0N2pYV0daUEpMaXpTamR6SEFJRkpGcEd2TFBVWmdQZXBiYXN6cz0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIzNzJmNjg5NTlmYWQ2NDBkOGEyMmQ3NTEzMWU3ODk0ZjE3MDViYWU5MzNkNjYzMzE3MjY5NjNmZiIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUE0NzM3MyIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzQ4OTk3MDAwLCJpYXQiOjE3NDg5Mjg4NzgsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc0ODkyODg3OCwic3ViIjoiYWNjZXNzX3Rva2VuIn0.R3-UNkIX_29uCijSiwbVtSPoLGsJju21Bs4T5uo10Zs"  # or your existing token
client_id = "FYZT8L00T9-100"
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)

def fetch_historical_data(symbol, start_date, end_date, interval="D"):
    data = {
        "symbol": symbol,
        "resolution": interval,
        "date_format": "1",  # Use "1" for string dates
        "range_from": start_date,
        "range_to": end_date,
        "cont_flag": "1"
    }
    response = fyers.history(data)
    if response['s'] == 'ok':
        df = pd.DataFrame(response['candles'], columns=['date', 'open', 'high', 'low', 'close', 'volume'])
        df['date'] = pd.to_datetime(df['date'], unit='s')
        df.set_index('date', inplace=True)
        return df
    else:
        print("Error fetching data:", response)
        return None

# Define the symbol and date range
symbol = "NSE:NIFTY50-INDEX"
start_date = "2024-05-01"
end_date = "2025-04-30"

# Fetch the data
df = fetch_historical_data(symbol, start_date, end_date)
if df is not None:
    #print(df.head())
    print(df)

                open      high       low     close  volume
date                                                      
2024-05-02  22567.85  22710.50  22567.85  22648.20       0
2024-05-03  22766.35  22794.70  22348.05  22475.85       0
2024-05-06  22561.60  22588.80  22409.45  22442.70       0
2024-05-07  22489.75  22499.05  22232.05  22302.50       0
2024-05-08  22231.20  22368.65  22185.20  22302.50       0
...              ...       ...       ...       ...     ...
2025-04-24  24277.90  24347.85  24216.15  24246.70       0
2025-04-25  24289.00  24365.45  23847.85  24039.35       0
2025-04-28  24070.25  24355.10  24054.05  24328.50       0
2025-04-29  24370.70  24457.65  24290.75  24335.95       0
2025-04-30  24342.05  24396.15  24198.75  24334.20       0

[248 rows x 5 columns]


In [2]:
def fetch_historical_data(symbol, start_date_str, end_date_str, interval="D"):
    import datetime
    all_candles = []
    
    # Convert string dates to datetime objects
    current_end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")
    overall_start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
    
    while current_end_date >= overall_start_date:
        # Calculate the start date for the current chunk
        chunk_start_date = current_end_date - datetime.timedelta(days=360)
        
        # Ensure the chunk_start_date does not go before the overall_start_date
        if chunk_start_date < overall_start_date:
            chunk_start_date = overall_start_date

        data = {
            "symbol": symbol,
            "resolution": interval,
            "date_format": "1",
            "range_from": chunk_start_date.strftime("%Y-%m-%d"),
            "range_to": current_end_date.strftime("%Y-%m-%d"),
            "cont_flag": "1"
        }
        
        print(f"Fetching data from {data['range_from']} to {data['range_to']}")
        response = fyers.history(data)

        if response['s'] == 'ok' and response.get('candles'):
            all_candles.extend(response['candles'])
        elif response['s'] == 'no_data':
            print(f"No data found for range: {data['range_from']} to {data['range_to']}")
            break 
        else:
            print("Error fetching data:", response)
            return None
        
        current_end_date = chunk_start_date - datetime.timedelta(days=1)
        
        if chunk_start_date == overall_start_date and current_end_date < overall_start_date:
            break 

    if not all_candles:
        print("No historical data fetched after multiple attempts.")
        return None

    df = pd.DataFrame(all_candles, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    df = df.sort_index()
    df = df[~df.index.duplicated(keep='first')]
    df = df.loc[start_date_str:end_date_str]

    return df

# The existing symbol, start_date, and end_date variables can be used directly
df = fetch_historical_data(symbol, start_date, end_date)
if df is not None:
    print(df)

# Define the symbol and date range
symbol = "NSE:NIFTY50-INDEX"
start_date = "2022-05-01"
end_date = "2025-04-30"

# Fetch the data
df = fetch_historical_data(symbol, start_date, end_date)
if df is not None:
    #print(df.head())
    print(df)

Fetching data from 2024-05-05 to 2025-04-30
Error fetching data: {'code': -16, 'message': 'Could not authenticate the user', 's': 'error'}
Fetching data from 2024-05-05 to 2025-04-30
Error fetching data: {'code': -16, 'message': 'Could not authenticate the user', 's': 'error'}


In [5]:
import calendar
import datetime
import pandas as pd

def fetch_historical_data(symbol: str, end_date_str: str, years_of_data: int, interval: str = "D"):
    """
    Fetches historical data for a symbol for a specified number of years ending on end_date_str.
    Data is fetched year by year.

    Args:
        symbol (str): The trading symbol (e.g., "NSE:NIFTY50-INDEX").
        end_date_str (str): The end date for the data in "YYYY-MM-DD" format.
        years_of_data (int): The number of past full years of data to fetch, plus the current year up to end_date.
                             Example: end_date_str="2025-04-30", years_of_data=4 means data from
                             2021-01-01 to 2021-12-31, ..., 2024-01-01 to 2024-12-31, 
                             and 2025-01-01 to 2025-04-30.
        interval (str): The data interval (e.g., "D" for daily).

    Returns:
        pandas.DataFrame: A DataFrame containing the historical data, or None if an error occurs.
    """
    #access_token = get_token()
    # The fyersModel should be imported. It might be fyersModel.FyersModel or similar
    # depending on the library version.
    #fyers = fyersModel(client_id=client_id, token=access_token)
    all_candles = []

    try:
        target_end_date_obj = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")
    except ValueError:
        print(f"Error: Invalid end_date_str format. Please use YYYY-MM-DD. Got: {end_date_str}")
        return None

    if not isinstance(years_of_data, int) or years_of_data < 0: # Allow 0 for current year only
        print("Error: years_of_data must be a non-negative integer.")
        return None

    # Calculate the range of years to fetch
    # If years_of_data = 4 and end_date is 2025-04-30, we fetch for 2021, 2022, 2023, 2024, and part of 2025.
    # The first year in the iteration will be target_end_date_obj.year - years_of_data.
    first_year_to_fetch = target_end_date_obj.year - years_of_data
    last_year_to_fetch = target_end_date_obj.year # This is the year of the end_date_str

    print(f"Preparing to fetch data for '{symbol}' from year {first_year_to_fetch} up to {end_date_str} (covering {years_of_data} prior full year(s) plus current year portion).")

    for year_iter in range(first_year_to_fetch, last_year_to_fetch + 1):
        current_chunk_start_date_str = f"{year_iter}-01-01"
        
        if year_iter < last_year_to_fetch:
            # For full years before the target_end_date_obj's year
            current_chunk_end_date_str = f"{year_iter}-12-31"
        else:
            # For the final year (which is target_end_date_obj.year), fetch up to target_end_date_obj
            current_chunk_end_date_str = target_end_date_obj.strftime("%Y-%m-%d")
        
        # Ensure the chunk_start_date is not later than chunk_end_date.
        if datetime.datetime.strptime(current_chunk_start_date_str, "%Y-%m-%d") > datetime.datetime.strptime(current_chunk_end_date_str, "%Y-%m-%d"):
            print(f"  Skipping data fetch for year {year_iter}: chunk start date {current_chunk_start_date_str} is after chunk end date {current_chunk_end_date_str}.")
            continue

        payload = {
            "symbol": symbol,
            "resolution": interval,
            "date_format": "1",  # "1" for epoch timestamp in response
            "range_from": current_chunk_start_date_str,
            "range_to": current_chunk_end_date_str,
            "cont_flag": "1"     # For continuous data for expired futures
        }
        
        print(f"Fetching data for '{symbol}' from {payload['range_from']} to {payload['range_to']}")
        response = fyers.history(data=payload) # Pass payload with keyword 'data='

        if response and response.get('s') == 'ok':
            if response.get('candles'):
                all_candles.extend(response['candles'])
                print(f"  Successfully fetched {len(response['candles'])} candles for {payload['range_from']} to {payload['range_to']}.")
            else:
                print(f"  No data in 'candles' (s='ok') for range: {payload['range_from']} to {payload['range_to']}.")
        elif response and response.get('s') == 'no_data':
            print(f"  API reported no data (s='no_data') for range: {payload['range_from']} to {payload['range_to']}. Message: {response.get('message')}")
        else:
            error_message = response.get('message', 'Unknown error') if response else "No response or malformed response from API"
            print(f"  Error fetching data for range {payload['range_from']} to {payload['range_to']}: {error_message}")
            # print(f"  Full response for error: {response}") # Uncomment for detailed debugging
            return None # Fail fast if any chunk results in an error
        
    if not all_candles:
        print(f"No historical data was fetched for '{symbol}' after all attempts for the period ending {end_date_str} covering {years_of_data} prior year(s).")
        return pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume']).set_index(pd.to_datetime([]))


    # Convert all collected candles to a DataFrame
    df = pd.DataFrame(all_candles, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    
    # Convert 'date' from epoch timestamp to datetime objects
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    
    # Sort by date as data from chunks might not be perfectly ordered
    df = df.sort_index()

    # Remove duplicate dates if any
    df = df[~df.index.duplicated(keep='first')]
    
    # Filter to ensure data is within the *overall* requested date range.
    # The overall_start_date is January 1st of the first_year_to_fetch.
    # The overall_end_date is the target_end_date_obj.
    overall_start_date_for_filter = datetime.datetime(first_year_to_fetch, 1, 1)
    
    # Apply the precise range filtering. Pandas slicing with datetime index includes both start and end.
    df = df.loc[overall_start_date_for_filter : target_end_date_obj]

    if df.empty:
        print(f"Dataframe is empty after processing for '{symbol}' ending {end_date_str} (requested {years_of_data} prior years).")
    else:
        print(f"Successfully processed data for '{symbol}'. Total records: {len(df)}. Date range in DataFrame: {df.index.min()} to {df.index.max()}")
    
    return df


# Example usage:
# Assume fyers and related initialization from your earlier code are already done.
symbol = "NSE:NIFTY50-INDEX"
end_date = "2025-04-30"
years_of_data = 4  # Fetch last 4 years (full years plus partial current year)

df = fetch_historical_data(symbol, end_date, years_of_data, interval="D")
if df is not None:
    print(df)

Preparing to fetch data for 'NSE:NIFTY50-INDEX' from year 2021 up to 2025-04-30 (covering 4 prior full year(s) plus current year portion).
Fetching data for 'NSE:NIFTY50-INDEX' from 2021-01-01 to 2021-12-31
  Successfully fetched 248 candles for 2021-01-01 to 2021-12-31.
Fetching data for 'NSE:NIFTY50-INDEX' from 2022-01-01 to 2022-12-31
  Successfully fetched 248 candles for 2022-01-01 to 2022-12-31.
Fetching data for 'NSE:NIFTY50-INDEX' from 2023-01-01 to 2023-12-31
  Successfully fetched 246 candles for 2023-01-01 to 2023-12-31.
Fetching data for 'NSE:NIFTY50-INDEX' from 2024-01-01 to 2024-12-31
  Successfully fetched 249 candles for 2024-01-01 to 2024-12-31.
Fetching data for 'NSE:NIFTY50-INDEX' from 2025-01-01 to 2025-04-30
  Successfully fetched 81 candles for 2025-01-01 to 2025-04-30.
Successfully processed data for 'NSE:NIFTY50-INDEX'. Total records: 1072. Date range in DataFrame: 2021-01-01 00:00:00 to 2025-04-30 00:00:00
                open      high       low     close  vol

In [ ]:
def fetch_historical_data(symbol, start_date_str, end_date_str, interval="D"):
    import datetime
    all_candles = []
    
    # Convert string dates to datetime objects
    current_end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")
    overall_start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
    
    while current_end_date >= overall_start_date:
        # Calculate the start date for the current chunk: go back 360 days
        chunk_start_date = current_end_date - datetime.timedelta(days=360)
        
        # Ensure the chunk_start_date does not go before the overall_start_date
        if chunk_start_date < overall_start_date:
            chunk_start_date = overall_start_date

        data = {
            "symbol": symbol,
            "resolution": interval,
            "date_format": "1",  # Use "1" for string dates
            "range_from": chunk_start_date.strftime("%Y-%m-%d"),
            "range_to": current_end_date.strftime("%Y-%m-%d"),
            "cont_flag": "1"
        }
        
        print(f"Fetching data from {data['range_from']} to {data['range_to']}")
        response = fyers.history(data)

        if response['s'] == 'ok' and response.get('candles'):
            all_candles.extend(response['candles'])
        elif response['s'] == 'no_data':
            print(f"No data found for range: {data['range_from']} to {data['range_to']}")
            break 
        else:
            print("Error fetching data:", response)
            return None
        
        # Set the end date for the next iteration to be one day before the current chunk's start date
        current_end_date = chunk_start_date - datetime.timedelta(days=1)
        
        # Safeguard to avoid infinite loops
        if chunk_start_date == overall_start_date and current_end_date < overall_start_date:
            break 

    if not all_candles:
        print("No historical data fetched after multiple attempts.")
        return None

    # Convert all collected candles to a DataFrame
    df = pd.DataFrame(all_candles, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    
    # Sort by date in ascending order
    df = df.sort_index()

    # Remove duplicate dates if any
    df = df[~df.index.duplicated(keep='first')]
    
    # Filter to include only the requested date range
    df = df.loc[start_date_str:end_date_str]

    return df

# Define the symbol and date range:
symbol = "NSE:NIFTY50-INDEX"
start_date = "2024-05-01"
end_date = "2025-04-30"

# Fetch the data:
df = fetch_historical_data(symbol, start_date, end_date)
if df is not None:
    print(df)

In [7]:
import numpy as np

def calculate_volatility(dates,closing_prices):
    if len(closing_prices) < 2:
        raise ValueError("At least two closing prices are required.")

    percentage_returns = []

    print("Daily Percentage Returns with Calculation Steps:\n")
    # # Calculate daily percentage returns
    # percentage_returns = [
    #     ((closing_prices[i] - closing_prices[i - 1]) / closing_prices[i - 1]) * 100
    #     for i in range(1, len(closing_prices))
    # ]
    for i in range(1, len(closing_prices)):
        today_price = closing_prices[i]
        yesterday_price = closing_prices[i - 1]
        pct_return = ((today_price - yesterday_price) / yesterday_price) * 100
        percentage_returns.append(pct_return)

        # Print date, formula and result
        print(f"{dates[i].date()}:\t(({today_price:.2f} - {yesterday_price:.2f}) / {yesterday_price:.2f}) * 100 = {pct_return:.4f}%")

    # print("LN")
    # for r in percentage_returns:
    #     print(r)

    # Calculate mean
    mean = np.mean(percentage_returns)
    print(f"Mean:  {mean:.6f}")

    # Calculate variance (sample variance)
    variance = np.var(percentage_returns, ddof=1)
    print(f"variance:  {variance:.6f}")

    # Daily standard deviation (volatility)
    daily_volatility = np.sqrt(variance)
    print(f"dailyVolatility:  {daily_volatility:.6f}")

    # Annualized volatility
    annualized_volatility = daily_volatility * np.sqrt(252)
    print(f"annualizedVolatility:  {annualized_volatility:.6f}")

    #last date close price i.e spot of the symbol
    spot = closing_prices[-1]
    print(f"spot:  {spot:.2f}")
    return annualized_volatility 

# Example usage
# closing_prices = [
#     558.77, 570.9, 576.85, 551.05, 557.05,
#     550.75, 544.4, 536.0, 548.65, 549.55,
#     551.4, 552.65, 548.05, 542.95
# ]
closing_prices = df["close"].tolist()
dates =df.index.tolist()

try:
    volatility = calculate_volatility(dates,closing_prices)
    print(f"Annualized Historical Volatility (%): {volatility:.4f}%")
except ValueError as e:
    print(f"Error: {e}")


Daily Percentage Returns with Calculation Steps:

2025-01-02:	((24188.65 - 23742.90) / 23742.90) * 100 = 1.8774%
2025-01-03:	((24004.75 - 24188.65) / 24188.65) * 100 = -0.7603%
2025-01-06:	((23616.05 - 24004.75) / 24004.75) * 100 = -1.6193%
2025-01-07:	((23707.90 - 23616.05) / 23616.05) * 100 = 0.3889%
2025-01-08:	((23688.95 - 23707.90) / 23707.90) * 100 = -0.0799%
2025-01-09:	((23526.50 - 23688.95) / 23688.95) * 100 = -0.6858%
2025-01-10:	((23431.50 - 23526.50) / 23526.50) * 100 = -0.4038%
2025-01-13:	((23085.95 - 23431.50) / 23431.50) * 100 = -1.4747%
2025-01-14:	((23176.05 - 23085.95) / 23085.95) * 100 = 0.3903%
2025-01-15:	((23213.20 - 23176.05) / 23176.05) * 100 = 0.1603%
2025-01-16:	((23311.80 - 23213.20) / 23213.20) * 100 = 0.4248%
2025-01-17:	((23203.20 - 23311.80) / 23311.80) * 100 = -0.4659%
2025-01-20:	((23344.75 - 23203.20) / 23203.20) * 100 = 0.6100%
2025-01-21:	((23024.65 - 23344.75) / 23344.75) * 100 = -1.3712%
2025-01-22:	((23155.35 - 23024.65) / 23024.65) * 100 = 0.567

Fetching the SPOT of the symbol

In [ ]:

from fyers_apiv3 import fyersModel
import json
from datetime import datetime, time
import pytz

def get_symbol_spot_price(client_id: str, access_token: str, symbol: str) -> float | None:
    """
    Fetches the current spot price (LTP) for a given symbol from Fyers.

    Args:
        client_id (str): Your Fyers API client ID.
        access_token (str): Your Fyers API access token.
        symbol (str): The Fyers symbol for which to fetch the spot price (e.g., "NSE:RELIANCE-EQ").

    Returns:
        float | None: The Last Traded Price (LTP) of the symbol if successful, otherwise None.
    """
    try:
        fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

        data = {"symbols": symbol}
        response = fyers.quotes(data=data)

        if response and response.get("code") == 200 and response.get("s") == "ok":
            symbol_data = response.get("d")
            if symbol_data:
                for item in symbol_data:
                    if item.get("symbol") == symbol:
                        # Extracting LTP, checking for common keys Fyers uses
                        # 'lp' (Last Price) is the most common and direct
                        # 'v' (value) might contain 'lp' or other details
                        ltp = item.get("lp")  # Direct Last Price
                        if ltp is None:
                            # If 'lp' is not directly present, check within 'v'
                            ltp_from_v = item.get("v", {}).get("lp")
                            if ltp_from_v is not None:
                                print(f"Found LTP for {symbol} under 'v.lp': {ltp_from_v}")
                                return ltp_from_v
                            else:
                                print(f"Warning: 'lp' or 'v.lp' not found for symbol '{symbol}' in response item.")
                                print(f"Full item data for '{symbol}': {json.dumps(item, indent=2)}")
                        else:
                            print(f"Found LTP for {symbol} under 'lp': {ltp}")
                            return ltp
                print(f"Error: Symbol '{symbol}' not found in the 'd' list of the Fyers API response.")
                print(f"Full Fyers API response for debugging: {json.dumps(response, indent=2)}")
                return None
            else:
                print("Error: 'd' (symbol data) list is empty in the Fyers API response.")
                print(f"Full Fyers API response for debugging: {json.dumps(response, indent=2)}")
                return None
        else:
            print(f"Error fetching quotes from Fyers API. Response: {json.dumps(response, indent=2)}")
            return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def is_nse_market_open():
    """
    Checks if NSE market is currently open.
    NSE trading hours (approx): Monday-Friday, 9:15 AM to 3:30 PM IST.
    """
    india_tz = pytz.timezone('Asia/Kolkata')
    now = datetime.now(india_tz)

    # Check if it's a weekday (Monday=0, Sunday=6)
    if now.weekday() >= 5:  # Saturday or Sunday
        print("Market is closed: Weekend.")
        return False

    market_open_time = time(9, 15)  # 9:15 AM IST
    market_close_time = time(15, 30) # 3:30 PM IST

    if market_open_time <= now.time() <= market_close_time:
        print("Market is currently open.")
        return True
    else:
        print(f"Market is closed: Outside trading hours (9:15 AM - 3:30 PM IST). Current time: {now.strftime('%H:%M:%S IST')}")
        return False

# --- How to use the function ---
if __name__ == "__main__":
    # Replace with your actual Fyers Client ID and Access Token
    your_client_id = "FYZT8L00T9-100"
    your_access_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb01abUlvdjlMVmdlUUVQU28xU3FnTXpXUXdaSGo0U3ZudzRUQTE4dExDd0RKa091SEJBZjBtS3JzR1FTVXhjeEY5VWJkbzU0cHFSNHloSlNxdnNONzNBQkhDTzZ5Zl9pUzJ6TVRrbWRWQjEzR19Zcz0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIzNzJmNjg5NTlmYWQ2NDBkOGEyMmQ3NTEzMWU3ODk0ZjE3MDViYWU5MzNkNjYzMzE3MjY5NjNmZiIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUE0NzM3MyIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzQ4MTMzMDAwLCJpYXQiOjE3NDgwODEwMzIsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc0ODA4MTAzMiwic3ViIjoiYWNjZXNzX3Rva2VuIn0.g7Dnnk57DGYCNmO4X2SokV9-LMh7JPXj4YGERbFDme8" # This token is likely expired or invalid if used outside its short validity period.

    # Check if market is open before trying to fetch live data
    if not is_nse_market_open():
        print("Cannot fetch live spot price as the market is closed.")
        # If market is closed, the API might not return live LTP or might return
        # old data or an empty 'd' array. This is a common reason for your error.
        print("Please try running this script during NSE market hours (Mon-Fri, 9:15 AM - 3:30 PM IST).")
    else:
        # Example symbols
        symbol_reliance = "NSE:RELIANCE-EQ"
        symbol_nifty = "NSE:NIFTY50-INDEX"
        symbol_invalid = "NSE:INVALIDSYMBOL-EQ" # Example of an invalid symbol

        print("\nAttempting to fetch spot prices:")

        # Fetch spot price for Reliance
        print(f"\n--- Fetching {symbol_reliance} ---")
        reliance_ltp = get_symbol_spot_price(your_client_id, your_access_token, symbol_reliance)
        if reliance_ltp is not None:
            print(f"Current Spot Price of {symbol_reliance}: {reliance_ltp}")
        else:
            print(f"Could not fetch spot price for {symbol_reliance}")

        print("-" * 30)

        # Fetch spot price for Nifty 50 Index
        print(f"\n--- Fetching {symbol_nifty} ---")
        nifty_ltp = get_symbol_spot_price(your_client_id, your_access_token, symbol_nifty)
        if nifty_ltp is not None:
            print(f"Current Spot Price of {symbol_nifty}: {nifty_ltp}")
        else:
            print(f"Could not fetch spot price for {symbol_nifty}")

        print("-" * 30)

        # Fetch spot price for an invalid symbol
        print(f"\n--- Fetching {symbol_invalid} (expected to fail) ---")
        invalid_ltp = get_symbol_spot_price(your_client_id, your_access_token, symbol_invalid)
        if invalid_ltp is not None:
            print(f"Current Spot Price of {symbol_invalid}: {invalid_ltp}")
        else:
            print(f"Could not fetch spot price for {symbol_invalid}")

Market is closed: Weekend.
Cannot fetch live spot price as the market is closed.
Please try running this script during NSE market hours (Mon-Fri, 9:15 AM - 3:30 PM IST).


In [1]:
import requests
import json
import logging

# Configure logging for better output (optional, but good practice)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_nifty_spot_ltp_sync() -> float | None:
    """
    Fetches the Last Traded Price (LTP) for NIFTY 50 from Groww's API synchronously.

    Returns:
        float | None: The LTP of NIFTY 50 if successful, otherwise None.
    """
    # Groww API endpoint for NIFTY 50
    full_url = "https://groww.in/v1/api/stocks_data/v1/accord_points/exchange/NSE/segment/CASH/latest_indices_ohlc/NIFTY"

    # Mimic browser headers to avoid potential blocking
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://groww.in/indices/nifty-50", # Referer for NIFTY 50 page
        "sec-ch-ua": '"Not A(Brand";v="8", "Chromium";v="97", "Microsoft Edge";v="97"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin"
    }

    timeout = 10 # seconds

    logger.info(f"Fetching NIFTY spot from Groww API: {full_url}")

    try:
        response = requests.get(full_url, headers=headers, timeout=timeout)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        data = response.json()

        # Log the full response for debugging (optional)
        logger.debug(f"Full Groww API response: {json.dumps(data, indent=2)}")

        # Extract the LTP (Last Traded Price)
        # The response has both 'close' and 'value'. 'value' is typically the most current.
        ltp = data.get("value")
        if ltp is None:
            ltp = data.get("close") # Fallback to 'close' if 'value' is not present

        if ltp is not None:
            try:
                return float(ltp)
            except ValueError:
                logger.error(f"Could not convert LTP '{ltp}' to float.")
                return None
        else:
            logger.error(f"Neither 'value' nor 'close' field found in Groww API response for NIFTY.")
            return None

    except requests.exceptions.HTTPError as errh:
        logger.error(f"HTTP Error occurred: {errh} - Response: {errh.response.text}")
        return None
    except requests.exceptions.ConnectionError as errc:
        logger.error(f"Error Connecting: {errc}")
        return None
    except requests.exceptions.Timeout as errt:
        logger.error(f"Timeout Error: {errt}")
        return None
    except requests.exceptions.RequestException as err:
        logger.error(f"Something went wrong with the request: {err}")
        return None
    except json.JSONDecodeError as e:
        logger.error(f"Failed to parse JSON response: {e}. Response text: {response.text}")
        return None
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        return None

# --- How to use the function ---
if __name__ == "__main__":
    nifty_spot = get_nifty_spot_ltp_sync()

    if nifty_spot is not None:
        print(f"Current NIFTY 50 Spot LTP (from Groww API): {nifty_spot}")
    else:
        print("Failed to fetch NIFTY 50 Spot LTP.")

2025-07-21 09:08:07,200 - INFO - Fetching NIFTY spot from Groww API: https://groww.in/v1/api/stocks_data/v1/accord_points/exchange/NSE/segment/CASH/latest_indices_ohlc/NIFTY


Current NIFTY 50 Spot LTP (from Groww API): 24968.4


In [21]:
from datetime import timedelta, datetime


def calculate_rolling_volatility(df: pd.DataFrame, calc_date: datetime) -> dict:
    """
    Calculate volatility for a 12-month window ending on calc_date
    """
    # Define the 12-month window
    window_end = calc_date - timedelta(days=1)
    window_start =  calc_date - pd.DateOffset(years=1)
    
    # Get data for this window
    window_data = df[window_start:window_end]
    
    if window_data.empty:
        raise ValueError(f"No data found for window {window_start} to {window_end}")

    # Calculate returns and volatility using existing function
    closing_prices = window_data['close'].tolist()
    dates = window_data.index.tolist()
    
    #result = calculate_volatility(dates, closing_prices)
    
    return {
        "calculation_date": calc_date.strftime("%Y-%m-%d"),
        "window_start": window_start.strftime("%Y-%m-%d"),
        "window_end": window_end.strftime("%Y-%m-%d"),
        "trading_days": len(window_data),
        #"volatility_stats": result
    }

# Example usage
calc_date = "2025-02-01"
calc_date = datetime.strptime(calc_date, "%Y-%m-%d")  # Convert string to datetime
volatility_stats = calculate_rolling_volatility(df, calc_date)
print(volatility_stats)


{'calculation_date': '2025-02-01', 'window_start': '2024-02-01', 'window_end': '2025-01-31', 'trading_days': 250}
